In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from mll_calc.mll_pred import format_XY, ratios, mll_testset

In [2]:
unc = 0.05
lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']
tamu_list = ['cs137/cs133', 'cs134/cs137', 'cs135/cs137', 'ba136/ba138',
             'sm150/sm149', 'sm152/sm149', 'eu154/eu153', 'pu240/pu239',
             'pu241/pu239', 'pu242/pu239']

In [3]:
train_db = '~/sims_n_results/sfcompo_compare_aug2020/van2ohi2_not-scaled_15nuc.pkl'
test_db = '~/sfcompo/format_clean/sfcompo_formatted.pkl'
XY = format_XY(train_db)
test = pd.read_pickle(test_db)

ratio_list = tamu_list
XY = ratios(XY, ratio_list, lbls)
test = ratios(test, ratio_list, lbls)

## Test sample from SFCOMPO that was simulated in ORIGEN

In [4]:
test = test.loc[test.index == 'OHI-2|17G|C5|89G03']
test

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor
OHI-2|17G|C5|89G03,0.0,0.022164,0.0,0.0,0.0,0.0,0.0,0.473684,0.281307,0.137568,pwr,0.0,1.6874,28717.0,Ohi-2_PWR


## 12 Sims from ORIGEN 

#### Only variations are: 
1. 12 cooling times, since that is unknown in SFCOMPO (ranging 1 min to 12 years)

#### OHI-2|17G|C5|89G0 Simulation Parameters:
* PWR: Ohi-2_PWR, so origen equivalent is W(17x17)
* Enrichment: 1.6874
* Burnup: 28717.0 MW*d/tHM
* Cooling time: 1 min --> 12 yrs
* U weight: 87 MT
* Thermal power: 3423 MW for cycles 5 & 6
* Avg power for ORIGEN is in MW/MTHM, so did 39 for sim

In [5]:
XY[3:6]

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor
3,0.955595,0.012955,0.680019,0.004306,21.074951,11.272189,0.061829,0.096108,0.015854,0.000622,pwr,0.0,1.69,28717.0,w17x17
4,0.945471,0.012847,0.681767,0.004377,19.148746,10.241935,0.061346,0.095540,0.015744,0.000618,pwr,10.0,1.69,28717.0,w17x17
5,0.936897,0.011890,0.685594,0.004478,19.063336,10.196253,0.060130,0.095540,0.015556,0.000618,pwr,100.0,1.69,28717.0,w17x17


## First, try to find best prediction via MLL

In [6]:
pred_df = mll_testset(XY[3:6].copy(), test, True, unc, lbls)

In [7]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
pred_df

,sim_idx,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,pred_idx,pred_ReactorType,pred_CoolingTime,pred_Enrichment,pred_Burnup,pred_OrigenReactor,MaxLogLL,MaxLLUnc,2ndMaxLogLL,2ndMaxLLUnc,CDF_LogLL_0.9998,CDF_LLUnc_0.9998,CDF_LogLL_0.9988,CDF_LLUnc_0.9988,CDF_LogLL_0.95,CDF_LLUnc_0.95,CDF_LogLL_0.9,CDF_LLUnc_0.9,CDF_LogLL_0.5,CDF_LLUnc_0.5,CDF_LogLL_0.1,CDF_LLUnc_0.1,CDF_LogLL_0.01,CDF_LLUnc_0.01
0,OHI-2|17G|C5|89G03,pwr,0.0,1.6874,28717.0,Ohi-2_PWR,3,pwr,0.0,1.69,28717.0,w17x17,-9.749165e+06,973423.950151,-9.866148e+06,985060.639334,-9.749212e+06,985061.576194,-9.749446e+06,985061.57432,-9.760863e+06,985061.482846,-9.772562e+06,985061.389122,-9.866148e+06,985060.639334,-9.867357e+06,975751.287987,-9.867629e+06,973656.683934


## Second, dig in and look at ratio diffs

In [8]:
def logpdf_calc(sim_row, test_sample, unc):
    y_sim = sim_row[test_sample>0].values.tolist()
    std = sim_row.multiply(unc)[test_sample>0].values.tolist()
    y_mes = test_sample[test_sample>0].values.tolist()
    logpdf_list = stats.norm.logpdf(y_sim, loc=y_mes, scale=std)
    return logpdf_list

In [9]:
test_sample = test.drop(columns=lbls).squeeze()
test_answer = test[lbls].squeeze()
sim_row = XY.loc[3].drop(lbls).squeeze()
logpdf = logpdf_calc(sim_row, test_sample, unc)
logpdf = pd.Series(logpdf, index=sim_row[test_sample>0].index, name='LogPDF')

In [10]:
df = pd.DataFrame([test_sample, sim_row])
df.loc['Abs Diff'] = np.abs(df.loc[3] - df.loc['OHI-2|17G|C5|89G03'])
df.loc['% Diff'] = df.loc['Abs Diff'] * 100 / df.loc['OHI-2|17G|C5|89G03']

In [11]:
df.loc['LogPDF'] = logpdf

In [12]:
df

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239
OHI-2|17G|C5|89G03,0.000000,0.022164,0.000000,0.000000,0.000000,0.000000,0.000000,0.473684,0.281307,1.375681e-01
3,0.955595,0.012955,0.680019,0.004306,21.074951,11.272189,0.061829,0.096108,0.015854,6.221622e-04
Abs Diff,0.955595,0.009209,0.680019,0.004306,21.074951,11.272189,0.061829,0.377576,0.265453,1.369459e-01
% Diff,inf,41.548685,inf,inf,inf,inf,inf,79.710511,94.364139,9.954774e+01
LogPDF,NaN,-94.631540,NaN,NaN,NaN,NaN,NaN,-3082.455170,-56062.866997,-9.689925e+06
